# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.16.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255
)

training_set = train_datagen.flow_from_directory(
    'dataset_3/Training/1.0_recyclable_Waste',
    target_size=(64, 64),
    batch_size=32,  # Increased batch size for faster GPU utilization
    class_mode='categorical'
)

Found 35986 images belonging to 32 classes.


In [4]:
training_set.class_indices


{'AluCan': 0,
 'Bottle': 1,
 'aluminum_food_cans': 2,
 'aluminum_soda_cans': 3,
 'cans': 4,
 'cardboard': 5,
 'cardboard_boxes': 6,
 'cardboard_packaging': 7,
 'carton': 8,
 'glass': 9,
 'glass_beverage_bottles': 10,
 'glass_cosmetic_containers': 11,
 'glass_food_jars': 12,
 'glassbottle': 13,
 'leaflet': 14,
 'magazines': 15,
 'metal': 16,
 'milkbox': 17,
 'newspaper': 18,
 'office_paper': 19,
 'paper': 20,
 'paper_cups': 21,
 'plastic_bags': 22,
 'plastic_cup_lids': 23,
 'plastic_detergent_bottles': 24,
 'plastic_food_containers': 25,
 'plastic_milk_bottles': 26,
 'plastic_shopping_bags': 27,
 'plastic_soda_bottles': 28,
 'plastic_trash_bags': 29,
 'plastic_water_bottles': 30,
 'steel_food_cans': 31}

### Preprocessing the Test set

In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
    'dataset_3/test/1.0_recyclable_Waste',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

Found 6096 images belonging to 32 classes.


## Part 2 - Building the CNN

In [6]:
# tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [7]:
# cnn = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]),
#     tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
#     tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
#     tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(units=128, activation='relu'),
#     tf.keras.layers.Dense(units=27, activation='softmax')
# ])

### Initialising the CNN

In [8]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=16, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.Conv2D(filters=16, kernel_size=3, activation='relu'))

C:\Users\Siddhant Jain\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [10]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [11]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [12]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [13]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [14]:
cnn.add(tf.keras.layers.Dense(units=32, activation='softmax'))

## Part 3 - Training the CNN

### Compiling the CNN

In [15]:
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [16]:
# def scheduler(epoch, lr):
#     if epoch < 10:
#         return lr
#     else:
#         return lr * tf.math.exp(-0.1)

# lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# # Train your model with adjusted batch size and mixed precision
# history = cnn.fit(
#     x=training_set,
#     validation_data=test_set,
#     epochs=20,
#     callbacks=[lr_scheduler]
# )

In [23]:
cnn.fit(x = training_set, validation_data = test_set, epochs =2)

Epoch 1/2
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 1367s 1s/step - accuracy: 0.9437 - loss: 0.1689 - val_accuracy: 0.8286 - val_loss: 1.4610
Epoch 2/2
1125/1125 ━━━━━━━━━━━━━━━━━━━━ 916s 813ms/step - accuracy: 0.9437 - loss: 0.1681 - val_accuracy: 0.8323 - val_loss: 1.5421


# Model Save

In [24]:
cnn.save('Recyclable_waste_model.h5')
cnn.save('Recyclable_waste_model.keras')

# Model Prediction checking by importing it

In [32]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the model
model = load_model('Recyclable_waste_model.keras')

# Function to preprocess the image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(64, 64))  # Adjust target_size to match your model input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)  # Adjust preprocessing if needed
    return img_array

# Function to make a prediction
def predict_image_class(model, img_path):
    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)
    return predicted_class

# Example usage
img_path = 'dataset_3/training/1.0_Recyclable_waste/plastic_cup_lids/Image_2.png'
predicted_class = predict_image_class(model, img_path)
print(f'Predicted class: {predicted_class[0]}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
Predicted class: 18


## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('dataset_3/training/1.0_recyclable_waste/cans/00000003.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices

predicted_index = np.argmax(result) 
print(predicted_index)

label_to_item = {
    0: 'Accessories',
    1: 'Battery',
    2: 'Camcorders',
    3: 'Cameras',
    4: 'Computers',
    5: 'Cooling Appliances',
    6: 'Dryers',
    7: 'Electronic Toys',
    8: 'Keyboard',
    9: 'Laptops',
    10: 'Microwave',
    11: 'Mobile',
    12: 'Modems',
    13: 'Monitors',
    14: 'Mouse',
    15: 'PCB',
    16: 'Player',
    17: 'Power Tools',
    18: 'Printer',
    19: 'Refrigerators',
    20: 'Routers',
    21: 'Scanners',
    22: 'Servers',
    23: 'Speakers',
    24: 'Television',
    25: 'Washing Machine',
    26: 'headphones'
}

# Getting the predicted item name
prediction = label_to_item.get(predicted_index, 'Unknown')

print(f"Prediction: {prediction}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
19
Prediction: Refrigerators
